In [ ]:
# ⚙️ PASO 1: Clonar repositorio desde GitHub
print("📦 Clonando repositorio...")
!rm -rf Botardo  # Limpiar si existe
!git clone https://github.com/TobiasActis/Botardo.git
%cd Botardo
!pwd
!ls -la

In [ ]:
# ⚙️ PASO 2: Instalar dependencias
print("📚 Instalando paquetes...")
!pip install -q pandas numpy matplotlib loguru ta requests

In [ ]:
# ⚙️ PASO 3: Descargar datos de Binance
print("📊 Descargando datos BTC 1m desde Binance...")
!python download_btc_futures_1m.py
print("✅ Datos descargados!")

In [ ]:
# 🚀 PASO 4: Ejecutar backtest completo 2024-2025
print("🚀 Iniciando backtest de 2 años...")
print("⏱️ Esto puede tardar 2-3 minutos\n")

!python backtest_superbot.py \
  --data_1m "data/BTCUSDT_1m_2024-01-01_to_now.csv" \
  --initial_capital 500 \
  --risk_per_trade 0.10 \
  --leverage 50 \
  --po3_min_rr 3.5 \
  --smc_standalone 8 \
  --start "2024-01-01" \
  --end "2025-12-09"

In [ ]:
# 📊 PASO 5: Mostrar gráfico de resultados
from IPython.display import Image, display
import os

if os.path.exists('superbot_backtest.png'):
    print("\n📈 GRÁFICO DE EQUITY:")
    display(Image('superbot_backtest.png', width=800))
else:
    print("⚠️ No se encontró el gráfico")

In [ ]:
# 📋 PASO 6: Analizar trades
import pandas as pd

# Cargar trades
trades = pd.read_csv('superbot_backtest_trades.csv')

print(f"\n📊 RESUMEN DE TRADES:")
print(f"Total trades: {len(trades)}")
print(f"Wins: {len(trades[trades['pnl'] > 0])}")
print(f"Losses: {len(trades[trades['pnl'] < 0])}")
print(f"Win Rate: {len(trades[trades['pnl'] > 0])/len(trades)*100:.2f}%")
print(f"\nTotal P&L: ${trades['pnl'].sum():.2f}")
print(f"Average Win: ${trades[trades['pnl'] > 0]['pnl'].mean():.2f}")
print(f"Average Loss: ${trades[trades['pnl'] < 0]['pnl'].mean():.2f}")
print(f"Best Trade: ${trades['pnl'].max():.2f}")
print(f"Worst Trade: ${trades['pnl'].min():.2f}")

print("\n📋 ÚLTIMOS 20 TRADES:")
print(trades[['timestamp', 'side', 'entry_price', 'exit_price', 'pnl', 'pnl_pct', 'exit_reason']].tail(20))

In [ ]:
# 💾 PASO 7: Descargar resultados a tu PC
from google.colab import files

print("💾 Descargando archivos...")
files.download('superbot_backtest.png')
files.download('superbot_backtest_trades.csv')
print("✅ Archivos descargados!")

---

## 🎯 ANÁLISIS POR MES

Ver performance mensual:

In [ ]:
# Análisis mensual
trades['timestamp'] = pd.to_datetime(trades['timestamp'])
trades['month'] = trades['timestamp'].dt.to_period('M')

monthly = trades.groupby('month').agg({
    'pnl': ['sum', 'count'],
    'pnl_pct': 'mean'
})

monthly.columns = ['Total P&L', 'Trades', 'Avg %']
print("\n📅 PERFORMANCE MENSUAL:")
print(monthly)

---

## ⚙️ CAMBIAR PARÁMETROS

Para probar con otros settings, edita el PASO 4:

```bash
--initial_capital 500     # Capital inicial
--risk_per_trade 0.10     # 10% de risk por trade
--leverage 50             # Leverage máximo
--po3_min_rr 3.5          # RR mínimo para PO3
--smc_standalone 8        # Confluence mínima SMC
--start "2024-01-01"      # Fecha inicio
--end "2025-12-09"        # Fecha fin
```